In [21]:
# imports
import torch
import torch.nn as nn
from torch.nn import (Conv2d, ReLU, MaxPool2d, Flatten, Linear, Sequential, 
                      CrossEntropyLoss)
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import os
import pickle
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_machine_learning.neural_networks import SamplerQNN
from qiskit_machine_learning.connectors import TorchConnector
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator


# Set device to XPU (Intel GPU)print(f"Device set to: {device}")

if torch.xpu.is_available():

    device = torch.device('xpu')

    print("✓ XPU (Intel GPU) is available!")
else:
    print("⚠ XPU not available, using CPU")
    device = torch.device('cpu')

✓ XPU (Intel GPU) is available!


In [22]:
# Select architectures to train on
SELECTED_CLASSES = ['dome(inner)', 'dome(outer)', 'gargoyle', 'stained_glass']

# Dataset path
DATA_DIR = '/home/advik/Quantum/Mini Project/architecture_dataset_32x32'

In [23]:
# Load dataset and create train/validation split
from sklearn.model_selection import train_test_split

class ArchitectureDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load images and labels
train_images = []
train_labels = []

train_dir = os.path.join(DATA_DIR, 'train')

for class_idx, class_name in enumerate(SELECTED_CLASSES):
    class_path = os.path.join(train_dir, class_name)
    
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path).convert('RGB')
        train_images.append(img)
        train_labels.append(class_idx)

print(f"Total images loaded: {len(train_images)}")
print(f"Classes: {SELECTED_CLASSES}")
print(f"Images per class: {[train_labels.count(i) for i in range(len(SELECTED_CLASSES))]}")

# Split train into train and validation (80/20)
train_imgs, val_imgs, train_lbls, val_lbls = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

print(f"\nTrain set: {len(train_imgs)} images")
print(f"Validation set: {len(val_imgs)} images")

# Create datasets
train_dataset = ArchitectureDataset(train_imgs, train_lbls, transform=transform)
val_dataset = ArchitectureDataset(val_imgs, val_lbls, transform=transform)

# Create dataloaders - REDUCED BATCH SIZE for quantum computing
batch_size = 8  # Smaller batches = faster quantum gradient computation
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

print(f"\nTrain batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")
print(f"⚡ Batch size reduced to {batch_size} for faster quantum gradient computation")

Total images loaded: 4324
Classes: ['dome(inner)', 'dome(outer)', 'gargoyle', 'stained_glass']
Images per class: [589, 1175, 1562, 998]

Train set: 3459 images
Validation set: 865 images

Train batches: 433
Validation batches: 109
⚡ Batch size reduced to 8 for faster quantum gradient computation


In [24]:
# Qiskit Machine Learning Quantum Circuit (Proper Implementation)
n_qubits = 4
n_layers = 2

def create_qiskit_qnn():
    """Create parameterized quantum circuit with Qiskit ML"""
    
    # Create parameters
    input_params = [Parameter(f'x_{i}') for i in range(n_qubits)]
    weight_params = [Parameter(f'θ_{i}') for i in range(n_qubits * n_layers)]
    
    # Build quantum circuit
    qc = QuantumCircuit(n_qubits)
    
    # Data encoding layer - RY rotations with input features
    for i in range(n_qubits):
        qc.ry(input_params[i], i)
    
    # Trainable parameterized layers
    param_idx = 0
    for layer in range(n_layers):
        # Rotation layer
        for i in range(n_qubits):
            qc.ry(weight_params[param_idx], i)
            param_idx += 1
        
        # Entanglement layer (circular)
        for i in range(n_qubits - 1):
            qc.cx(i, i + 1)
        qc.cx(n_qubits - 1, 0)
    
    # Create SamplerQNN with gradient support
    qnn = SamplerQNN(
        circuit=qc,
        input_params=input_params,
        weight_params=weight_params,
        input_gradients=True  # Enable gradient computation
    )
    
    # Wrap with TorchConnector for PyTorch integration
    quantum_layer = TorchConnector(qnn)
    
    return quantum_layer

# Create the quantum layer
quantum_layer = create_qiskit_qnn()

print("✓ Qiskit Machine Learning quantum layer created!")
print(f"Qubits: {n_qubits}, Layers: {n_layers}, Parameters: {n_qubits * n_layers}")
print(f"TorchConnector: ENABLED - Full gradient support")
print(f"Device: {device}")

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


✓ Qiskit Machine Learning quantum layer created!
Qubits: 4, Layers: 2, Parameters: 8
TorchConnector: ENABLED - Full gradient support
Device: xpu


In [25]:
# Hybrid Quantum-Classical Model with Qiskit ML
class HybridQNN(nn.Module):
    def __init__(self, quantum_layer, n_qubits=4, n_classes=4):
        super(HybridQNN, self).__init__()
        
        # Simple CNN feature extractor (32x32x3 -> n_qubits)
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8
            nn.Flatten(),
            nn.Linear(32 * 8 * 8, 64),
            nn.ReLU(),
            nn.Linear(64, n_qubits)  # Map to n_qubits inputs
        )
        
        # Quantum layer (TorchConnector handles gradients automatically!)
        self.quantum_layer = quantum_layer
        
        # Classical classifier (SamplerQNN outputs 2^n_qubits probabilities)
        self.classifier = nn.Sequential(
            nn.Linear(2**n_qubits, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, n_classes)
        )
        
    def forward(self, x):
        # Extract classical features
        features = self.feature_extractor(x)  # (batch, n_qubits)
        features = torch.tanh(features) * np.pi  # Scale to quantum rotation range
        
        # Process through quantum layer (fully differentiable!)
        quantum_outputs = self.quantum_layer(features)
        
        # Final classification
        output = self.classifier(quantum_outputs)
        
        return output

# Create model with Qiskit ML quantum layer
model = HybridQNN(quantum_layer=quantum_layer, n_qubits=n_qubits, n_classes=len(SELECTED_CLASSES))
model.to(device)
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Model device: {next(model.parameters()).device}")
print("\n✓ All components are differentiable - gradients will flow!")

HybridQNN(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=2048, out_features=64, bias=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=4, bias=True)
  )
  (quantum_layer): TorchConnector()
  (classifier): Sequential(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=32, out_features=4, bias=True)
  )
)

Total parameters: 137168
Model device: xpu:0

✓ All components are differentiable - gradients will flow!


In [26]:
# Training setup
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(loader):
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        if batch_idx % 5 == 0:
            print(f'Batch {batch_idx}/{len(loader)}, Loss: {loss.item():.4f}')
    
    accuracy = 100. * correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    accuracy = 100. * correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

print("Training functions ready!")

Training functions ready!


In [ ]:
# Train the model with live plotting
n_epochs = 5  # Reduced for quantum - each epoch takes longer
train_losses = []
val_losses = []
train_accs = []
val_accs = []

print(f"Starting training for {n_epochs} epochs...\n")
print("⚠️ Note: Quantum gradient computation is slow (parameter shift rule)")
print("   Each batch processes quantum circuits multiple times for gradients\n")

# Setup live plotting
%matplotlib inline
from IPython import display

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for epoch in range(n_epochs):
    print(f"Epoch {epoch+1}/{n_epochs}")
    print("-" * 50)
    
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    
    # Validate
    val_loss, val_acc = validate(model, val_loader, criterion)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    
    print(f"\nEpoch {epoch+1} Summary:")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    print("=" * 50 + "\n")
    
    # Update plots in real-time
    ax1.clear()
    ax2.clear()
    
    # Plot loss
    ax1.plot(range(1, epoch+2), train_losses, 'b-o', label='Train Loss', linewidth=2, markersize=6)
    ax1.plot(range(1, epoch+2), val_losses, 'r-s', label='Val Loss', linewidth=2, markersize=6)
    ax1.set_xlabel('Epoch', fontsize=11)
    ax1.set_ylabel('Loss', fontsize=11)
    ax1.set_title('Training and Validation Loss', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot accuracy
    ax2.plot(range(1, epoch+2), train_accs, 'b-o', label='Train Acc', linewidth=2, markersize=6)
    ax2.plot(range(1, epoch+2), val_accs, 'r-s', label='Val Acc', linewidth=2, markersize=6)
    ax2.set_xlabel('Epoch', fontsize=11)
    ax2.set_ylabel('Accuracy (%)', fontsize=11)
    ax2.set_title('Training and Validation Accuracy', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Update display
    display.clear_output(wait=True)
    display.display(fig)

print(f"\n🎯 Final Results:")
print(f"Best Val Accuracy: {max(val_accs):.2f}%")
print(f"Final Train Accuracy: {train_accs[-1]:.2f}%")
print(f"Final Val Accuracy: {val_accs[-1]:.2f}%")

Starting training for 10 epochs...

⚠️ Note: Quantum gradient computation is slow (parameter shift rule)
   Each batch processes quantum circuits multiple times for gradients

Epoch 1/10
--------------------------------------------------
Batch 0/433, Loss: 1.3701
Batch 5/433, Loss: 1.3872
Batch 10/433, Loss: 1.3818
Batch 15/433, Loss: 1.4029
Batch 20/433, Loss: 1.3390
Batch 25/433, Loss: 1.3124
Batch 30/433, Loss: 1.2938
Batch 35/433, Loss: 1.3936
Batch 40/433, Loss: 1.3575
Batch 45/433, Loss: 1.3574
Batch 50/433, Loss: 1.4073
Batch 55/433, Loss: 1.3483
Batch 60/433, Loss: 1.3086
Batch 65/433, Loss: 1.3903
Batch 70/433, Loss: 1.2880
Batch 75/433, Loss: 1.3881
Batch 80/433, Loss: 1.2352
Batch 85/433, Loss: 1.2849
Batch 90/433, Loss: 1.2003
Batch 95/433, Loss: 1.2094
Batch 100/433, Loss: 1.2703
Batch 105/433, Loss: 1.2248
Batch 110/433, Loss: 1.4097
Batch 115/433, Loss: 1.3290
Batch 120/433, Loss: 1.2333
Batch 125/433, Loss: 1.1338
Batch 130/433, Loss: 1.1821
Batch 135/433, Loss: 1.1812
